In [42]:
import seaborn as sns
sns.set(rc={'axes.facecolor': 'dimgrey', 'grid.color': 'lightgrey'})
import pandas as pd
import matplotlib.pyplot as plt
from torch_geometric.nn import Node2Vec
import torch
from tqdm.notebook import tqdm
# Now train the DT classifier
# Load libraries
import pandas as pd
from torch_geometric.data import Data
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM

In [43]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cpu


In [189]:
%run utils.ipynb
%run gcn_model.ipynb
%run GAT_model.ipynb

In [190]:
# class 1 is illicit, 2 is licit, unknown is unknown
labels = pd.read_csv("data/elliptic_bitcoin_dataset/elliptic_txs_classes.csv")
edges = pd.read_csv("data/elliptic_bitcoin_dataset/elliptic_txs_edgelist.csv")
nodes = pd.read_csv("data/elliptic_bitcoin_dataset/elliptic_txs_features.csv", header=None)
print('done')

done


In [191]:
indexes = labels.index[labels["class"] != "unknown"].tolist()
labels_new = labels[labels["class"] != "unknown"]
nodes_new =  nodes.take(indexes)
edges_new = edges.loc[edges["txId1"].isin(labels_new["txId"])].loc[edges["txId2"].isin(labels_new["txId"])]
nodes_new.shape, labels_new.shape, edges_new.shape

((46564, 167), (46564, 2), (36624, 2))

In [198]:
data, graph_info = time_step_split(nodes_new, edges_new, labels_new, device, train_lt = 49, val_lt = 49, test_lt = 49)
# data = data['train']

print(data)

{'train': [Data(x=[2147, 165], edge_index=[2, 1924], y=[2147]), Data(x=[1117, 165], edge_index=[2, 858], y=[1117]), Data(x=[1279, 165], edge_index=[2, 727], y=[1279]), Data(x=[1440, 165], edge_index=[2, 1169], y=[1440]), Data(x=[1882, 165], edge_index=[2, 1491], y=[1882]), Data(x=[485, 165], edge_index=[2, 209], y=[485]), Data(x=[1203, 165], edge_index=[2, 858], y=[1203]), Data(x=[1165, 165], edge_index=[2, 1044], y=[1165]), Data(x=[778, 165], edge_index=[2, 484], y=[778]), Data(x=[972, 165], edge_index=[2, 538], y=[972]), Data(x=[696, 165], edge_index=[2, 477], y=[696]), Data(x=[506, 165], edge_index=[2, 446], y=[506]), Data(x=[809, 165], edge_index=[2, 564], y=[809]), Data(x=[417, 165], edge_index=[2, 350], y=[417]), Data(x=[618, 165], edge_index=[2, 446], y=[618]), Data(x=[530, 165], edge_index=[2, 334], y=[530]), Data(x=[811, 165], edge_index=[2, 673], y=[811]), Data(x=[389, 165], edge_index=[2, 275], y=[389]), Data(x=[745, 165], edge_index=[2, 585], y=[745]), Data(x=[900, 165], ed

In [199]:
def trainGCN(data_train, data):

    GCN = GNN_Example(node_dim=data['train'][0].x.shape[1], edge_dim=1, output_dim=2, hidden_dim=256, n_gnn_layers=3, K=2, dropout_rate=0.1).to(device)

    lr = 1e-3
    epochs = 50
    # weights for classes
    weight = torch.tensor([0.5, 0.5]).to(device)
    optimizer = torch.optim.Adam(GCN.parameters(), lr=lr)
    loss_fn = torch.nn.NLLLoss(weight)

    best_model = None
    best_valid_auc = 0
    best_result = None
    losses = []

    for epoch in range(1, 1 + epochs):
        # train with random split
        loss = train(GCN, data_train, optimizer, loss_fn, device)
        losses.append(loss)
        result = test(GCN, data, device)
        train_acc, val_acc, test_acc, train_auc, val_auc, test_auc = result
        if val_auc > best_valid_auc:
            best_valid_auc = val_auc
            best_model = copy.deepcopy(GCN)
            best_result = [train_acc, val_acc, test_acc, train_auc, val_auc, test_auc]
        if epoch % 10 == 0:
            print('Epoch: {:02},'.format(epoch),
                  'Loss:{:.4f}'.format(loss),
                  'Train:\n{}\n'.format(train_acc),
                  'Train_auc_roc: {}'.format(train_auc),
                  '\n\n'
                  'Valid:\n{}\n'.format(val_acc),
                  'Val_auc_roc: {}'.format(val_auc),
                  '\n\n'
                  'Test:\n{}\n'.format(test_acc),
                  'Test_auc_roc: {}'.format(test_auc),
                  '\n'
                  )

    return best_result

In [200]:
# cross validation
for idx in range(49):
    data['test'] = data['train'][idx]
    data['val'] =  data['train'][idx]
    train_data = [x for i, x in enumerate(data['train']) if i != idx]
    result = trainGCN(train_data, data)

TypeError: unhashable type: 'dict'